Title: Named Entity Recognition

Description:
In this NER-focused project, you will design and develop a custom Named Entity Recognition (NER) system for text analysis. Named Entity Recognition involves identifying and classifying specific entities, such as names, dates, locations, and more, within unstructured text data. Your project will offer a versatile NER solution that will work well on the provided dataset.

Dataset: Named Entity Recognition (NER) Corpus (kaggle.com)

https://www.kaggle.com/datasets/naseralqaydeh/named-entity-recognition-ner-corpus

In [1]:
!date

Mon Apr  7 13:39:18 UTC 2025


In [2]:
import warnings,gc,sys
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import tensorflow as tf
from IPython.display import display, HTML
import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split, GridSearchCV

from tqdm import tqdm

2025-04-07 13:39:19.370168: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-07 13:39:19.371934: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-07 13:39:19.379722: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-07 13:39:19.393769: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744033159.414839   12487 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744033159.42

In [3]:
# !pip install kagglehub

In [4]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "ner.csv"

# Load the latest version
NER_df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "naseralqaydeh/named-entity-recognition-ner-corpus",
  file_path,

)

# print("First 5 records:", df.head())

In [5]:
display(NER_df.head(2).T)

,0,1
Sentence #,Sentence: 1,Sentence: 2
Sentence,Thousands of demonstrators have marched throug...,Families of soldiers killed in the conflict jo...
POS,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ..."
Tag,"['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


Essential info about entities:

- geo = Geographical Entity
- org = Organization
- per = Person
- gpe = Geopolitical Entity
- tim = Time indicator
- art = Artifact
- eve = Event
- nat = Natural Phenomenon

### 1. Preprocess

In [6]:
print(NER_df.shape)

(47959, 4)


In [7]:
print(NER_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47959 entries, 0 to 47958
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Sentence #  47959 non-null  object
 1   Sentence    47959 non-null  object
 2   POS         47959 non-null  object
 3   Tag         47959 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB
None


In [8]:
NER_df.dropna(inplace=True)
NER_df.drop(columns=["Sentence #","POS"],inplace=True)

In [9]:
print(NER_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47959 entries, 0 to 47958
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  47959 non-null  object
 1   Tag       47959 non-null  object
dtypes: object(2)
memory usage: 749.5+ KB
None


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(NER_df["Sentence"], NER_df["Tag"], shuffle=True,test_size=0.20, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, shuffle=True,test_size=0.50, random_state=42)

In [23]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(lower=False,oov_token="UNK")
tokenizer.fit_on_texts(X_train)

In [22]:
X_train.reset_index(drop=True)

0        The 58-year-old former analyst says he provide...
1        But he said he will not accept any Pakistani p...
2        The Swiss star was upset Wednesday by German T...
3        After taking office in 2004 , the SPENCER gove...
4        Israeli soldiers have killed two Palestinians ...
                               ...                        
38362    However , poverty , illiteracy , and unemploym...
38363    The report was published two days after human ...
38364    In a separate incident , kidnappers released a...
38365    Those groups were shut down by U.S. officials ...
38366    A statement on the Web site of the Health Depa...
Name: Sentence, Length: 38367, dtype: object

In [24]:
train_inputs = tokenizer.texts_to_sequences(X_train)
test_inputs = tokenizer.texts_to_sequences(X_test)
val_inputs = tokenizer.texts_to_sequences(X_val)

In [28]:
print(f"Number of unique tokens:\t{len(tokenizer.word_index)}")

Number of unique tokens:	28761


In [29]:
train_tags = set([val for sublist in y_train for val in sublist])
test_tags = set([val for sublist in y_test for val in sublist])
val_tags = set([val for sublist in y_val for val in sublist])

print("Unique NER tags in train set: ",train_tags)
print("Unique NER tags in test set: ",test_tags)
print("Unique NER tags in test set: ",val_tags)

Unique NER tags in train set:  {'t', 'g', 'O', 'a', 'n', 'I', 'i', "'", ' ', 'e', 'r', ',', '-', '[', 'v', 'o', 'm', 'p', 'B', ']'}
Unique NER tags in test set:  {'t', 'g', 'O', 'a', 'n', 'i', 'I', "'", ' ', 'e', 'r', ',', '-', '[', 'v', 'o', 'm', 'p', 'B', ']'}
Unique NER tags in test set:  {'t', 'g', 'O', 'a', 'n', 'I', 'i', "'", ' ', 'e', 'r', ',', '-', '[', 'v', 'o', 'm', 'p', 'B', ']'}


In [20]:
# NER_df['Sentence'] = tokenizer.texts_to_sequences(["I love deep learning"])
NER_df["Sequences"] = NER_df["Sentence"].apply(lambda x: tokenizer.texts_to_sequences([x])[0])
display(NER_df.head(2).T)

,0,1
Sentence,Thousands of demonstrators have marched throug...,Families of soldiers killed in the conflict jo...
Tag,"['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
Sequences,"[1152, 4, 1202, 15, 1893, 237, 457, 5, 555, 2,...","[8701, 4, 145, 36, 3, 2, 527, 1134, 2, 677, 53..."


In [13]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense,Input,Bidirectional,LSTM,Embedding
from keras.models import Model
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import EarlyStopping

ModuleNotFoundError: No module named 'keras.preprocessing.text'

In [32]:
# train_targets = list(df_train.Tag.values)
# test_targets = list(df_test.Tag.values)

tokenizer = Tokenizer(lower=False,oov_token="UNK")
tokenizer.fit_on_texts(NER_df['Sentence'],)

# train_inputs = tokenizer.texts_to_sequences(df_train['Sentence'])
# test_inputs = tokenizer.texts_to_sequences(df_test['Sentence'])

NameError: name 'Tokenizer' is not defined

In [31]:
NER_df["Tokenized_Sentence"] = NER_df["Sentence"].apply(lambda x: tokenizer.texts_to_sequences([x]))
# NER_df["Tokenized_Sentence"] = NER_df["Sentence"].apply(lambda x: tokenizer.texts_to_sequences([x])[0])

NameError: name 'tokenizer' is not defined

In [9]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

2025-04-07 13:12:49.924812: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [10]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18137355800593050408
 xla_global_id: -1]

In [11]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [12]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [13]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [14]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [15]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.7193052 , -0.47969562,  0.32077536,  0.70289576, -0.04301793,
        -0.67874795, -0.23602428,  0.31352717, -0.5399128 ,  0.35995692]],
      dtype=float32)

In [16]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [17]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [18]:
tf.device('/device:GPU:0')

In [19]:
with tf.device('/device:GPU:0'):
    model.fit(x_train, y_train, epochs=500, batch_size=32)

Epoch 1/500
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8551 - loss: 0.4949
Epoch 2/500
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9535 - loss: 0.1564
Epoch 3/500
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9668 - loss: 0.1071
Epoch 4/500
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9737 - loss: 0.0829
Epoch 5/500
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9762 - loss: 0.0744
Epoch 6/500
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9804 - loss: 0.0638
Epoch 7/500
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9813 - loss: 0.0562
Epoch 8/500
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9837 - loss: 0.0493
Epoch 9/500
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9849 - loss: 0.0455
Epoch 10/500
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9862 - loss: 0.0420
Epoch 11/500
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9863 - loss: 0.0402
Epoch 12/500
1875/1

KeyboardInterrupt: 

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

In [ ]:
predictions = model(x_test[:1]).numpy()

313/313 - 1s - 4ms/step - accuracy: 0.9763 - loss: 0.0764


[0.07639684528112411, 0.9763000011444092]

In [ ]:
predictions